# **ANALYSE ET PREDICTIONS SUR LES ELECTIONS PRESIDENTIELLES DE LA FRANCE**

In [59]:
import pandas as pd

# Charger les fichiers CSV
data_idf = pd.read_csv("elections-france-presidentielles-2022-1er-tour-par-bureau-de-vote-idf.csv", sep=";")
print('dataset elections: \n',data_idf.head())
data_security = pd.read_csv("donnee-dep-data.gouv-2023-geographie2024-produit-le2024-07-05.csv", sep=";")
print('dataset securité: \n',data_security.head())
# Afficher les dimensions des DataFrames
print("Dimensions de data_idf :", data_idf.shape)
print("Dimensions de data_security_idf :", data_security.shape)


dataset elections: 
    Code du département Libellé du département  Code de la circonscription  \
0                   77         Seine-et-Marne                           4   
1                   77         Seine-et-Marne                           2   
2                   77         Seine-et-Marne                           3   
3                   77         Seine-et-Marne                           2   
4                   77         Seine-et-Marne                           2   

  Libellé de la circonscription  Code de la commune Libellé de la commune  \
0          4ème circonscription               77002               Amillis   
1          2ème circonscription               77003            Amponville   
2          3ème circonscription               77004              Andrezel   
3          2ème circonscription               77009               Arville   
4          2ème circonscription               77011           Aufferville   

  Code du b.vote  Inscrits  Abstentions  % Abs/Ins  .

In [34]:
# Afficher les colonnes du DataFrame
data_idf.columns

Index(['Code du département', 'Libellé du département',
       'Code de la circonscription', 'Libellé de la circonscription',
       'Code de la commune', 'Libellé de la commune', 'Code du b.vote',
       'Inscrits', 'Abstentions', '% Abs/Ins', 'Votants', '% Vot/Ins',
       'Blancs', '% Blancs/Ins', '% Blancs/Vot', 'Nuls', '% Nuls/Ins',
       '% Nuls/Vot', 'Exprimés', '% Exp/Ins', '% Exp/Vot', 'N°Panneau', 'Sexe',
       'Nom', 'Prénom', 'Voix', '% Voix/Ins', '% Voix/Exp',
       'Code Officiel EPCI', 'Nom Officiel EPCI', 'Code Officiel Région',
       'Nom Officiel Région', 'scrutin_code', 'location', 'lib_du_b_vote'],
      dtype='object')

In [35]:
# Afficher les colonnes du DataFrame
data_security.columns

Index(['classe', 'annee', 'Code.département', 'Code.région', 'unité.de.compte',
       'millPOP', 'millLOG', 'faits', 'POP', 'LOG', 'tauxpourmille'],
      dtype='object')

In [64]:
from operator import le
import pandas as pd

# 1. Vérifier les valeurs manquantes
print("Valeurs manquantes par colonne :")
print(data_idf.isnull().sum())# Supprimer les colonnes où toutes les valeurs sont NaN
print("Valeurs manquantes par colonne pour le dataset security :")
print(data_security.isnull().sum())
#verifier les doublons dans les datasets
print("Doublons dans le dataset election :")
print(data_idf.duplicated().sum())
print("Doublons dans le dataset security :")
print(data_security.duplicated().sum())
#supprimer les colonnes où toutes les valeurs sont NaN
data_idf_cleaned = data_idf.dropna(axis=1, how='all')
# Supprimer les lignes où toutes les valeurs sont NaN
data_idf_cleaned = data_idf_cleaned.dropna(axis=0, how='all')
# 3. Supprimer les doublons
data_idf_cleaned = data_idf_cleaned.drop_duplicates()
data_security = data_security.drop_duplicates()
# 4. Renommer les colonnes (exemple : suppression des espaces)
data_idf_cleaned.columns = data_idf_cleaned.columns.str.strip()  # Supprimer les espaces avant et après les noms des colonnes
print("Colonnes nettoyées :", data_idf_cleaned.columns)
# Renommer les colonnes du dataset security
data_security.columns = data_security.columns.str.replace(".", "_")  # Remplacer les points par des underscores
print("Colonnes nettoyées securite:", data_security.columns)
#renommer les colonnes du dataset data_idf_cleaned
data_idf_cleaned = data_idf_cleaned.rename(columns={"Code du département": "Code_département"})
print("Colonnes nettoyées election :", data_idf_cleaned.columns) 
#enregister les données nettoyées
data_idf_cleaned.to_csv("elections-france-presidentielles-2022-1er-tour-par-bureau-de-vote-idf-cleaned.csv", index=False)
data_security.to_csv("donnee-dep-data.gouv-2023-geographie2024-produit-le2024-07-05-cleaned.csv", index=False)

Valeurs manquantes par colonne :
Code du département                  0
Libellé du département               0
Code de la circonscription           0
Libellé de la circonscription        0
Code de la commune                   0
Libellé de la commune                0
Code du b.vote                       0
Inscrits                             0
Abstentions                          0
% Abs/Ins                            0
Votants                              0
% Vot/Ins                            0
Blancs                               0
% Blancs/Ins                         0
% Blancs/Vot                         0
Nuls                                 0
% Nuls/Ins                           0
% Nuls/Vot                           0
Exprimés                             0
% Exp/Ins                            0
% Exp/Vot                            0
N°Panneau                            0
Sexe                                 0
Nom                                  0
Prénom                         

In [ ]:
#gérer les outliers doublons
import pandas as pd

# Step 1: Charger le dataset avec le bon séparateur
file_path = r"C:\Users\cound\OneDrive\Desktop\EIDF\analyse-election-presidentielle-2022-france\elections-france-presidentielles-2022-1er-tour-par-bureau-de-vote-idf.csv"
data = pd.read_csv(file_path, sep=';', encoding='utf-8')

# Step 2: Supprimer les doublons
data = data.drop_duplicates()  # Supprime les lignes en double

# Step 3: Vérifier les outliers et les gérer (par exemple, la colonne 'Voix')
# Fonction pour supprimer les outliers en utilisant la méthode IQR
def remove_outliers_iqr(df, column):
    # Calcul des quartiles Q1 (25%) et Q3 (75%)
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    
    # Calcul de l'IQR
    IQR = Q3 - Q1
    
    # Définir les limites inférieure et supérieure pour les outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Filtrer les lignes qui ne sont pas des outliers
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Exemple : appliquer la fonction pour gérer les outliers dans la colonne 'Voix'
if 'Voix' in data.columns:
    data = remove_outliers_iqr(data, 'Voix')  # Remplacez 'Voix' par la colonne de votre choix

# Step 4: Vérifier la qualité des données
print("Missing values per column:")
print(data.isnull().sum())

# Step 5: Standardiser les colonnes de type chaîne (par exemple, enlever les espaces inutiles)
data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Step 6: Vérification du résultat
print("\nDonnées après gestion des outliers et des doublons:")
print(data.head())

# Step 7: Sauvegarder le dataset nettoyé dans un nouveau fichier CSV
cleaned_file_path = r"C:\Users\cound\OneDrive\Desktop\EIDF\analyse-election-presidentielle-2022-france\cleaned_data.csv"
data.to_csv(cleaned_file_path, index=False, encoding='utf-8')

print("\nLe fichier nettoyé a été sauvegardé sous : ", cleaned_file_path)


In [ ]:
# Normalisation des colonnes numériques
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
num_columns = data_idf_cleaned.select_dtypes(include=['float64', 'int64']).columns
data_idf_cleaned[num_columns] = scaler.fit_transform(data_idf_cleaned[num_columns])


